# Import

In [ ]:
# binary classification spot check script
import warnings
from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Original

In [ ]:
# load the dataset, returns X and y elements
def load_dataset():
	return make_classification(n_samples=1000, n_classes=2, random_state=1)

# create a dict of standard models to evaluate {name:object}
def define_models(models=dict()):
	# linear models
	models['logistic'] = LogisticRegression()
	alpha = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
	for a in alpha:
		models['ridge-'+str(a)] = RidgeClassifier(alpha=a)
	models['sgd'] = SGDClassifier(max_iter=1000, tol=1e-3)
	models['pa'] = PassiveAggressiveClassifier(max_iter=1000, tol=1e-3)
	# non-linear models
	n_neighbors = range(1, 21)
	for k in n_neighbors:
		models['knn-'+str(k)] = KNeighborsClassifier(n_neighbors=k)
	models['cart'] = DecisionTreeClassifier()
	models['extra'] = ExtraTreeClassifier()
	models['svml'] = SVC(kernel='linear')
	models['svmp'] = SVC(kernel='poly')
	c_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
	for c in c_values:
		models['svmr'+str(c)] = SVC(C=c)
	models['bayes'] = GaussianNB()
	# ensemble models
	n_trees = 100
	models['ada'] = AdaBoostClassifier(n_estimators=n_trees)
	models['bag'] = BaggingClassifier(n_estimators=n_trees)
	models['rf'] = RandomForestClassifier(n_estimators=n_trees)
	models['et'] = ExtraTreesClassifier(n_estimators=n_trees)
	models['gbm'] = GradientBoostingClassifier(n_estimators=n_trees)
	print('Defined %d models' % len(models))
	return models

# no transforms pipeline
def pipeline_none(model):
	return model

# standardize transform pipeline
def pipeline_standardize(model):
	steps = list()
	# standardization
	steps.append(('standardize', StandardScaler()))
	# the model
	steps.append(('model', model))
	# create pipeline
	pipeline = Pipeline(steps=steps)
	return pipeline

# normalize transform pipeline
def pipeline_normalize(model):
	steps = list()
	# normalization
	steps.append(('normalize', MinMaxScaler()))
	# the model
	steps.append(('model', model))
	# create pipeline
	pipeline = Pipeline(steps=steps)
	return pipeline

# standardize and normalize pipeline
def pipeline_std_norm(model):
	steps = list()
	# standardization
	steps.append(('standardize', StandardScaler()))
	# normalization
	steps.append(('normalize', MinMaxScaler()))
	# the model
	steps.append(('model', model))
	# create pipeline
	pipeline = Pipeline(steps=steps)
	return pipeline

# evaluate a single model
def evaluate_model(X, y, model, folds, metric, pipe_func):
	# create the pipeline
	pipeline = pipe_func(model)
	# evaluate model
	scores = cross_val_score(pipeline, X, y, scoring=metric, cv=folds, n_jobs=-1)
	return scores

# evaluate a model and try to trap errors and and hide warnings
def robust_evaluate_model(X, y, model, folds, metric, pipe_func):
	scores = None
	try:
		with warnings.catch_warnings():
			warnings.filterwarnings("ignore")
			scores = evaluate_model(X, y, model, folds, metric, pipe_func)
	except:
		scores = None
	return scores

# evaluate a dict of models {name:object}, returns {name:score}
def evaluate_models(X, y, models, pipe_funcs, folds=10, metric='accuracy'):
	results = dict()
	for name, model in models.items():
		# evaluate model under each preparation function
		for i in range(len(pipe_funcs)):
			# evaluate the model
			scores = robust_evaluate_model(X, y, model, folds, metric, pipe_funcs[i])
			# update name
			run_name = str(i) + name
			# show process
			if scores is not None:
				# store a result
				results[run_name] = scores
				mean_score, std_score = mean(scores), std(scores)
				print('>%s: %.3f (+/-%.3f)' % (run_name, mean_score, std_score))
			else:
				print('>%s: error' % run_name)
	return results

# print and plot the top n results
def summarize_results(results, maximize=True, top_n=10):
	# check for no results
	if len(results) == 0:
		print('no results')
		return
	# determine how many results to summarize
	n = min(top_n, len(results))
	# create a list of (name, mean(scores)) tuples
	mean_scores = [(k,mean(v)) for k,v in results.items()]
	# sort tuples by mean score
	mean_scores = sorted(mean_scores, key=lambda x: x[1])
	# reverse for descending order (e.g. for accuracy)
	if maximize:
		mean_scores = list(reversed(mean_scores))
	# retrieve the top n for summarization
	names = [x[0] for x in mean_scores[:n]]
	scores = [results[x[0]] for x in mean_scores[:n]]
	# print the top n
	print()
	for i in range(n):
		name = names[i]
		mean_score, std_score = mean(results[name]), std(results[name])
		print('Rank=%d, Name=%s, Score=%.3f (+/- %.3f)' % (i+1, name, mean_score, std_score))
	# boxplot for the top n
	pyplot.boxplot(scores, labels=names)
	_, labels = pyplot.xticks()
	pyplot.setp(labels, rotation=90)
	pyplot.savefig('spotcheck.png')

# load dataset
X, y = load_dataset()
# get model list
models = define_models()
# define transform pipelines
pipelines = [pipeline_none, pipeline_standardize, pipeline_normalize, pipeline_std_norm]
# evaluate models
results = evaluate_models(X, y, models, pipelines)
# summarize results
summarize_results(results)

# Modified

In [ ]:
import pandas as pd
from sklearn.neural_network import MLPClassifier

In [ ]:
# load the dataset, returns X and y elements
def load_dataset():
    d = pd.read_csv('source/d_num.csv')
    d_values = d.values
    return d_values[:,1:], d_values[:,:1].ravel()

# create a dict of standard models to evaluate {name:object}
def define_models(models=dict()):
    # linear models
     
    # non-linear model
#     iter_times = [500, 1000, 2000]
#     for i in iter_times:
#         models['MLP'] = MLPClassifier(max_iter=i, random_state=5)
#     models['cart'] = DecisionTreeClassifier()
#     models['extra'] = ExtraTreeClassifier()
#     models['svmp'] = SVC(kernel='poly')
#     c_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
#     for c in c_values:
#         models['svmr'+str(c)] = SVC(C=c)
        
    # ensemble models
    n_trees = [10, 30, 50 , 100]
    models['ada'] = AdaBoostClassifier(n_estimators=n_trees, random_state=5)
    models['bag'] = BaggingClassifier(n_estimators=n_trees, n_jobs=-1, random_state=5)
    models['rf'] = RandomForestClassifier(n_estimators=n_trees, n_jobs=-1, random_state=5)
    models['et'] = ExtraTreesClassifier(n_estimators=n_trees, n_jobs=-1, random_state=5)
    models['gbm'] = GradientBoostingClassifier(n_estimators=n_trees, random_state=5)
    print('Defined %d models' % len(models))
    return models

# no transforms pipeline
def pipeline_none(model):
	return model

# standardize transform pipeline
def pipeline_standardize(model):
	steps = list()
	# standardization
	steps.append(('standardize', StandardScaler()))
	# the model
	steps.append(('model', model))
	# create pipeline
	pipeline = Pipeline(steps=steps)
	return pipeline

# normalize transform pipeline
def pipeline_normalize(model):
	steps = list()
	# normalization
	steps.append(('normalize', MinMaxScaler()))
	# the model
	steps.append(('model', model))
	# create pipeline
	pipeline = Pipeline(steps=steps)
	return pipeline

# standardize and normalize pipeline
def pipeline_std_norm(model):
	steps = list()
	# standardization
	steps.append(('standardize', StandardScaler()))
	# normalization
	steps.append(('normalize', MinMaxScaler()))
	# the model
	steps.append(('model', model))
	# create pipeline
	pipeline = Pipeline(steps=steps)
	return pipeline

# evaluate a single model
def evaluate_model(X, y, model, folds, metric, pipe_func):
	# create the pipeline
	pipeline = pipe_func(model)
	# evaluate model
	scores = cross_val_score(pipeline, X, y, scoring=metric, cv=folds, n_jobs=-1)
	return scores

# evaluate a model and try to trap errors and and hide warnings
def robust_evaluate_model(X, y, model, folds, metric, pipe_func):
	scores = None
	try:
		with warnings.catch_warnings():
			warnings.filterwarnings("ignore")
			scores = evaluate_model(X, y, model, folds, metric, pipe_func)
	except:
		scores = None
	return scores

# evaluate a dict of models {name:object}, returns {name:score}
def evaluate_models(X, y, models, pipe_funcs, folds=10, metric='recall'):
	results = dict()
	for name, model in models.items():
		# evaluate model under each preparation function
		for i in range(len(pipe_funcs)):
			# evaluate the model
			scores = robust_evaluate_model(X, y, model, folds, metric, pipe_funcs[i])
			# update name
			run_name = str(i) + name
			# show process
			if scores is not None:
				# store a result
				results[run_name] = scores
				mean_score, std_score = mean(scores), std(scores)
				print('>%s: %.3f (+/-%.3f)' % (run_name, mean_score, std_score))
			else:
				print('>%s: error' % run_name)
	return results

# print and plot the top n results
def summarize_results(results, maximize=True, top_n=10):
	# check for no results
	if len(results) == 0:
		print('no results')
		return
	# determine how many results to summarize
	n = min(top_n, len(results))
	# create a list of (name, mean(scores)) tuples
	mean_scores = [(k,mean(v)) for k,v in results.items()]
	# sort tuples by mean score
	mean_scores = sorted(mean_scores, key=lambda x: x[1])
	# reverse for descending order (e.g. for accuracy)
	if maximize:
		mean_scores = list(reversed(mean_scores))
	# retrieve the top n for summarization
	names = [x[0] for x in mean_scores[:n]]
	scores = [results[x[0]] for x in mean_scores[:n]]
	# print the top n
	print()
	for i in range(n):
		name = names[i]
		mean_score, std_score = mean(results[name]), std(results[name])
		print('Rank=%d, Name=%s, Score=%.3f (+/- %.3f)' % (i+1, name, mean_score, std_score))
	# boxplot for the top n
	pyplot.boxplot(scores, labels=names)
	_, labels = pyplot.xticks()
	pyplot.setp(labels, rotation=90)
	pyplot.savefig('spotcheck.png')

# load dataset
X, y = load_dataset()
# get model list
models = define_models()
# define transform pipelines
pipelines = [pipeline_none, pipeline_standardize, pipeline_normalize, pipeline_std_norm]
# evaluate models
results = evaluate_models(X, y, models, pipelines)
# summarize results
summarize_results(results)

In [ ]:
# example of grid searching key hyperparameters for GradientBoostingClassifier
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
# define dataset
X, y = make_blobs(n_samples=1000, centers=2, n_features=100, cluster_std=20)
# define models and parameters
model = GradientBoostingClassifier()
n_estimators = [10, 100, 1000]
learning_rate = [0.001, 0.01, 0.1]
subsample = [0.5, 0.7, 1.0]
max_depth = [3, 7, 9]
# define grid search
grid = dict(learning_rate=learning_rate, n_estimators=n_estimators, subsample=subsample, max_depth=max_depth)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Randomized Search CV

In [ ]:
print('Running RandomizedSearchCV')
# default RFClassifier but set random_state = 0 to keep consistent results
MOD = RandomForestClassifier(random_state=0) 
#Implement RandomSearchCV
# Number of trees in random forest [100,150,...,500]
n_estimators = [int(x) for x in np.arange(start = 100, stop = 501, step = 50)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.arange(start = 20, stop = 101, step = 20)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
scoreFunction = {"recall": "recall"}
# run a RandomizedSearchCV with 3 folds and 25 iterations 
random_search = RandomizedSearchCV(MOD,
                                   param_distributions = random_grid,
                                   n_iter = 25,
                                   scoring = scoreFunction,               
                                   refit = "recall",
                                   return_train_score = False,
                                   random_state = 0,
                                   verbose = 2,
                                   cv = 3,
                                   n_jobs = -1) 

#trains and optimizes the model
random_search.fit(X_train80, y_train80)

print('Finished RandomizedSearchCV ')

In [12]:
import pandas as pd
import numpy as np

## Load Data

In [3]:
# load the dataset, returns X and y elements
def load_dataset():
    d = pd.read_csv('source/d_num.csv')
    d_values = d.values
    x, y = d_values[:,1:], d_values[:,:1].ravel()
    return x, y

## Define Models

In [4]:
# create a dict of standard models to evaluate {name:object}
def define_models(models=list()):
    # linear
    i = 1000
    models.append(('LR', LogisticRegression(solver='saga', max_iter=i, class_weight='balanced', random_state=5))) # note: `max_iter` from 1000 to 10000 due to convergence issues
    # non-linear
    models.append(('DT', DecisionTreeClassifier(random_state=5)))
    i = 1000
    models.append(('MLP_' + str(i), MLPClassifier(max_iter=i, random_state=5)))
    # ensemble
    t = 800
    models.append(('ADA_' + str(t), AdaBoostClassifier(n_estimators=t, random_state=5))) 
    models.append(('GB_' + str(t), GradientBoostingClassifier(n_estimators=t, random_state=5))) # note: `max_iter` from 100 to 1000 due to convergence issues
    print(f'Defined {len(models)} models.')
    return models

## Define Resamplers

[Source: How to Combine Oversampling and Undersampling for Imbalanced Classification, *Machine Learning Mastery*](https://machinelearningmastery.com/combine-oversampling-and-undersampling-for-imbalanced-classification/)
<br>[Source: Undersampling Algorithms for Imbalanced Classification, *Machine Learning Mastery*](https://machinelearningmastery.com/undersampling-algorithms-for-imbalanced-classification/)

In [5]:
# create a dict of standard models to evaluate {name:object}
def define_resamplers(resamplers=list()):
    resamplers.append(('SM', SMOTE(random_state=5, n_jobs=-1)))
    resamplers.append(('SM_TM', SMOTETomek(tomek=TomekLinks(sampling_strategy='majority')))) 
    resamplers.append(('SM_ENN', SMOTEENN(enn=EditedNearestNeighbours(sampling_strategy='majority')))) 
    print(f'Defined {len(resamplers)} resamplers.')
    return resamplers

## Build Pipelines

In [6]:
# normalize and standardize transform pipeline : RS + Resampling
def pipeline_RS_SS_resample(model, resampler):
    steps = list()
    # normalization
    steps.append(('RS', RobustScaler()))
    # standardization
    steps.append(('SS', StandardScaler()))
    # the resampler
    steps.append(('RSP', resampler))
    # the model
    steps.append(('MOD', model))
    # create pipeline
    pipeline = Pipeline(steps=steps)
    return pipeline

## Try One Model

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold, cross_validate
from sklearn.metrics import make_scorer,fbeta_score

In [20]:
X, y = load_dataset()
models = define_models()
resamplers = define_resamplers()
for i in resamplers:
    print(i)
for i in models:
    print(i)

In [22]:
print(models[4][1])
print(resamplers[2][1])

In [34]:
model = models[4][1]
resampler = resamplers[2][1]
pipeline = pipeline_RS_SS_resample(model, resampler)
scores = cross_val_score(pipeline, X, y, scoring='recall', cv=10, n_jobs=-1)

In [38]:
# define cv method
n_splits = 10
n_repeats = 5
cv = RepeatedStratifiedKFold(
    n_splits=n_splits, n_repeats=n_repeats, random_state=1)  

# define performance metrics
scoring = {
    'accuracy':'accuracy', 'precision':'precision', 'recall':'recall', 'f1':'f1', 
    'f2':make_scorer(fbeta_score, beta=2)} # dict val = scorer fct or predefined metric str  

# evaluate result
result = cross_validate(
    pipeline, X, y, cv=cv, 
    scoring=scoring, return_train_score=True, n_jobs=-1)

# make a summary table
df = pd.DataFrame(
    (k, np.nanmean(v), np.nanstd(v)) for k,v in result.items()
    ).rename({0:'metric', 1:'mean', 2:'std'}, axis=1
            ).set_index('metric')
df.index.name = None
df.columns = pd.MultiIndex.from_product([['mod_disp_name'],df.columns])

In [39]:
df

mod_disp_name      
                         mean   std
fit_time               201.53 24.98
score_time               0.08  0.04
test_accuracy            0.97  0.00
train_accuracy           1.00  0.00
test_precision           0.93  0.02
train_precision          1.00  0.00
test_recall              0.91  0.02
train_recall             0.99  0.00
test_f1                  0.92  0.01
train_f1                 0.99  0.00
test_f2                  0.91  0.02
train_f2                 0.99  0.00